Se realiza la carga de las librerías requeridas en el proceso.

In [36]:
from sklearn.metrics import classification_report
from numbers import Number
import numpy as np
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import sklearn
import tensorflow as tf
import os
import zipfile
from google.colab import files
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import pandas as pd
import json
import random
from PIL import Image
from collections import Counter
from google.colab import drive
from tensorflow.keras.models import load_model
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss
import mlflow.keras

## Conexión MlFlow

En esta sección se genera la conexión a MlFlow mediante *ngrok* con el fin de poder tener acceso a la interfaz gráfica de la herramienta.

In [2]:
!pip install mlflow requests
import mlflow
import os
import pandas as pd
from IPython.display import display

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.0/25.0 MB 46.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.6/147.6 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.1/106.1 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.4/84.4 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.9/202.9 kB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 6.4 MB/s eta 0:00:00
  Attempting unins

In [3]:
command = """
mlflow server \
        --backend-store-uri sqlite:///tracking.db \
        --default-artifact-root file:mlruns \
        -p 5000 &
"""
get_ipython().system_raw(command)

In [4]:
!pip install pyngrok

In [5]:
token = "2g3elaQlzc0qSVIqOJWR4rApSoT_2TSiaCTXVnsD2LzY2LLYk" # Agregue el token dentro de las comillas
os.environ["NGROK_TOKEN"] = token

In [6]:
!ngrok authtoken $NGROK_TOKEN

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [7]:
from pyngrok import ngrok
ngrok.connect(5000, "http")


<NgrokTunnel: "https://f03a-34-125-253-33.ngrok-free.app" -> "http://localhost:5000">

In [8]:
mlflow.set_tracking_uri("http://localhost:5000")

In [9]:
exp_id = mlflow.create_experiment(name="proyecto_mdls6_v1", artifact_location="mlruns/")

## Publicación Model_128N

In [37]:
#Definimos el modelo en keras
conv_net = tf.keras.models.Sequential()

#Capa de entrada
width  =X_train.shape[1]
height  =X_train.shape[2]
conv_net.add(tf.keras.layers.Input(shape=(width, height, 3)))

#Primer bloque
conv_net.add(tf.keras.layers.Conv2D(
    filters = 8, kernel_size = 5, activation = 'relu'
))
conv_net.add(tf.keras.layers.MaxPooling2D(pool_size = (2,2)))

#Segundo bloque
conv_net.add(tf.keras.layers.Conv2D(
    filters = 16,
    kernel_size = 4,
    activation = 'relu'
))
conv_net.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))

#Tercer bloque
conv_net.add(tf.keras.layers.Conv2D(
    filters = 32,
    kernel_size = 3,
    activation = 'relu'
))
conv_net.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))

#Cuarto bloque
conv_net.add(tf.keras.layers.Conv2D(
    filters = 64,
    kernel_size = 3,
    activation = 'relu'
))
conv_net.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))

#Quinto bloque
conv_net.add(tf.keras.layers.Conv2D(
    filters = 128,
    kernel_size = 3,
    activation = 'relu'
))
conv_net.add(tf.keras.layers.MaxPooling2D(pool_size = (2,2)))

#Sexto bloque
conv_net.add(tf.keras.layers.Conv2D(
    filters = 256,
    kernel_size = 2,
    activation = 'relu'
))
conv_net.add(tf.keras.layers.MaxPooling2D(pool_size = (2,2)))

#capa flatten, que transforma cualquier arreglo multidimensional en un vector unidimensional.
conv_net.add(tf.keras.layers.Flatten())

#####Clasificador
# Capa densa intermedia
conv_net.add(tf.keras.layers.Dense(units=128, activation='relu'))

# Capa densa intermedia
conv_net.add(tf.keras.layers.Dense(units=128, activation='relu'))

# Agregamos dropout para regularización
conv_net.add(tf.keras.layers.Dropout(0.2))

# Capa de salida
conv_net.add(tf.keras.layers.Dense(units=18, #Número de categorías
                                   activation='softmax')) #Clasificación multiclase


In [38]:
conv_net.load_weights("best_weights_128N.h5")

y_pred = conv_net.predict(X_test)
y_pred_classes = y_pred.argmax(axis=1)  # Obtener la clase con la mayor probabilidad

# Obtener las etiquetas reales
y_true = Y_test_1.argmax(axis=1)


10/10 [==============================] - 5s 475ms/step


In [39]:
report_df = pd.DataFrame(classification_report(y_true, y_pred_classes, output_dict=True)).transpose()
accuracy = accuracy_score(y_true, y_pred_classes)
loss = log_loss(y_true, y_pred)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [40]:
run = mlflow.start_run(
        run_name="model_128N", experiment_id=exp_id
        )

model=conv_net

precision= report_df.loc["macro avg"][0]
recall= report_df.loc["macro avg"][1]
f1_score= report_df.loc["macro avg"][2]

mlflow.keras.log_model(model, "model")

mlflow.log_metrics({
        "accuracy":accuracy,
        "loss":loss,
        "precision": precision,
        "recall": recall,
        "f1_score": f1_score
        })

mlflow.end_run()

2024/05/23 00:09:16 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.
/usr/local/lib/python3.10/dist-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


## Publicación Model_256N

In [41]:
#Definimos el modelo en keras
conv_net = tf.keras.models.Sequential()

#Capa de entrada
width  =X_train.shape[1]
height  =X_train.shape[2]
conv_net.add(tf.keras.layers.Input(shape=(width, height, 3)))

#Primer bloque
conv_net.add(tf.keras.layers.Conv2D(
    filters = 8, kernel_size = 5, activation = 'relu'
))
conv_net.add(tf.keras.layers.MaxPooling2D(pool_size = (2,2)))

#Segundo bloque
conv_net.add(tf.keras.layers.Conv2D(
    filters = 16,
    kernel_size = 4,
    activation = 'relu'
))
conv_net.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))

#Tercer bloque
conv_net.add(tf.keras.layers.Conv2D(
    filters = 32,
    kernel_size = 3,
    activation = 'relu'
))
conv_net.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))

#Cuarto bloque
conv_net.add(tf.keras.layers.Conv2D(
    filters = 64,
    kernel_size = 3,
    activation = 'relu'
))
conv_net.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))

#Quinto bloque
conv_net.add(tf.keras.layers.Conv2D(
    filters = 128,
    kernel_size = 3,
    activation = 'relu'
))
conv_net.add(tf.keras.layers.MaxPooling2D(pool_size = (2,2)))

#capa flatten, que transforma cualquier arreglo multidimensional en un vector unidimensional.
conv_net.add(tf.keras.layers.Flatten())

#####Clasificador
# Capa densa intermedia
conv_net.add(tf.keras.layers.Dense(units=256, activation='relu'))

# Capa densa intermedia
conv_net.add(tf.keras.layers.Dense(units=256, activation='relu'))

# Agregamos dropout para regularización
conv_net.add(tf.keras.layers.Dropout(0.2))

# Capa de salida
conv_net.add(tf.keras.layers.Dense(units=18, #Número de categorías
                                   activation='softmax')) #Clasificación multiclase

In [42]:
conv_net.load_weights("best_weights_256N.h5")

y_pred = conv_net.predict(X_test)
y_pred_classes = y_pred.argmax(axis=1)  # Obtener la clase con la mayor probabilidad

# Obtener las etiquetas reales
y_true = Y_test_1.argmax(axis=1)


10/10 [==============================] - 4s 411ms/step


In [43]:
report_df = pd.DataFrame(classification_report(y_true, y_pred_classes, output_dict=True)).transpose()
accuracy = accuracy_score(y_true, y_pred_classes)
loss = log_loss(y_true, y_pred)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [44]:
run = mlflow.start_run(
        run_name="model_256N", experiment_id=exp_id
        )

model=conv_net

precision= report_df.loc["macro avg"][0]
recall= report_df.loc["macro avg"][1]
f1_score= report_df.loc["macro avg"][2]

mlflow.keras.log_model(model, "model")

mlflow.log_metrics({
        "accuracy":accuracy,
        "loss":loss,
        "precision": precision,
        "recall": recall,
        "f1_score": f1_score
        })

mlflow.end_run()

2024/05/23 00:10:25 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


## Publicación Model_512N

In [45]:
#Definimos el modelo en keras
conv_net = tf.keras.models.Sequential()

#Capa de entrada
width  =X_train.shape[1]
height  =X_train.shape[2]
conv_net.add(tf.keras.layers.Input(shape=(width, height, 3)))

#Primer bloque
conv_net.add(tf.keras.layers.Conv2D(
    filters = 8, kernel_size = 6, activation = 'relu'
))
conv_net.add(tf.keras.layers.MaxPooling2D(pool_size = (2,2)))

#Segundo bloque
conv_net.add(tf.keras.layers.Conv2D(
    filters = 16,
    kernel_size = 5,
    activation = 'relu'
))
conv_net.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))

#Tercer bloque
conv_net.add(tf.keras.layers.Conv2D(
    filters = 32,
    kernel_size = 4,
    activation = 'relu'
))
conv_net.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))

#Cuarto bloque
conv_net.add(tf.keras.layers.Conv2D(
    filters = 64,
    kernel_size = 3,
    activation = 'relu'
))
conv_net.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))


#capa flatten, que transforma cualquier arreglo multidimensional en un vector unidimensional.
conv_net.add(tf.keras.layers.Flatten())

#####Clasificador
# Capa densa intermedia
conv_net.add(tf.keras.layers.Dense(units=512, activation='relu'))

# Capa densa intermedia
conv_net.add(tf.keras.layers.Dense(units=512, activation='relu'))

# Agregamos dropout para regularización
conv_net.add(tf.keras.layers.Dropout(0.2))

# Capa de salida
conv_net.add(tf.keras.layers.Dense(units=18, #Número de categorías
                                   activation='softmax')) #Clasificación multiclase


In [46]:
conv_net.load_weights("best_weights_512N.h5")

y_pred = conv_net.predict(X_test)
y_pred_classes = y_pred.argmax(axis=1)  # Obtener la clase con la mayor probabilidad

# Obtener las etiquetas reales
y_true = Y_test_1.argmax(axis=1)


10/10 [==============================] - 5s 468ms/step


In [47]:
report_df = pd.DataFrame(classification_report(y_true, y_pred_classes, output_dict=True)).transpose()
accuracy = accuracy_score(y_true, y_pred_classes)
loss = log_loss(y_true, y_pred)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [48]:
run = mlflow.start_run(
        run_name="model_512N", experiment_id=exp_id
        )

model=conv_net

precision= report_df.loc["macro avg"][0]
recall= report_df.loc["macro avg"][1]
f1_score= report_df.loc["macro avg"][2]

mlflow.keras.log_model(model, "model")

mlflow.log_metrics({
        "accuracy":accuracy,
        "loss":loss,
        "precision": precision,
        "recall": recall,
        "f1_score": f1_score
        })

mlflow.end_run()

2024/05/23 00:10:52 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.
